# 0. Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import warnings

# Filter out the specific UserWarnings
warnings.filterwarnings("ignore", category=UserWarning, message="A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy")
warnings.filterwarnings("ignore", category=UserWarning, message="unable to load libtensorflow_io_plugins.so")
warnings.filterwarnings("ignore", category=UserWarning, message="file system plugins are not loaded")

In [ ]:
# Hugging Face library
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification

In [ ]:
# Hugging Face library
from datasets import Dataset, DatasetDict

In [ ]:
# Accuracy metrics and model selection from Scikit-Learn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf

from tensorflow.keras.optimizers import AdamW


from tensorflow.keras.losses import SparseCategoricalCrossentropy

from tensorflow.keras.layers import Input, Dense

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler

from tensorflow.keras.metrics import SparseCategoricalAccuracy

# 1. Load Dataset

In [ ]:
# Create a function to import the data from xlxs format

def load_data(file_path):
    return pd.read_excel(file_path, header=None,names = ['ID','DATE','CHANNEL','text','sentiment','emotion','info'], skiprows=1)

df_path = '/kaggle/input/peaks-sentiment-emotion/Peaks_sentiment_emotion.xlsx'

df = load_data(df_path)

In [ ]:
pd.set_option('display.max_colwidth', 150)
df.head()

In [ ]:
df = df[~((df['sentiment'] == 'UNPREDICTABLE') | (df['emotion'] == 'UNPREDICTABLE'))]

df = df[(df['sentiment'] != ' - ') & (df['emotion'] != ' - ') & (df['info'] != ' - ')]

df = df[~((df['CHANNEL'] != 'TWITTER'))]

df.reset_index(drop=True, inplace=True)

In [ ]:
# Remove rows with NaN values
df = df.dropna()

In [ ]:
# Check whether there are duplicates in the dataframe

print(df.shape)

duplicates = df.duplicated()

duplicate_count = duplicates.value_counts()
print(duplicate_count)

duplicate_lines = df[duplicates].index
for line in duplicate_lines:
    print(f"Duplicate row at line {line}:")
    print(df.loc[line])

df = df.drop_duplicates()

print(df.shape)

In [ ]:
df.head()

## 1.2 Preparing datasets

In [ ]:
# Since I'm gonna use the sparse categorical cross entropy loss, I map the labels to integers

encoded_dict_emotion = {'TRISTEZZA':0, 'GIOIA':1, 'AMORE':2, 'RABBIA':3, 'PAURA':4, 'SORPRESA':5, 'NEUTRA':6}

df['emotion_label'] = df['emotion'].apply(lambda x:encoded_dict_emotion[x])

def label2id_emotion(label):
    if isinstance(label, list):
        return [encoded_dict_emotion[label] for label in label]
    else:
        return encoded_dict_emotion[label]
    
def id2label_emotion(id):
    encoded_dict_inv_emotion = {v: k for k, v in encoded_dict_emotion.items()}
    
    if isinstance(id, list):
        return [encoded_dict_inv_emotion[i] for i in id]
    else:
        return encoded_dict_inv_emotion[id]

In [ ]:
encoded_dict_sentiment = {'NEG':0, 'NEU':1, 'POS':2}

df['sentiment_label'] = df['sentiment'].apply(lambda x:encoded_dict_sentiment[x])

def label2id_sentiment(label):
    if isinstance(label, list):
        return [encoded_dict_sentiment[label] for label in label]
    else:
        return encoded_dict_sentiment[label]

def id2label_sentiment(id):
    encoded_dict_inv_sentiment = {v: k for k, v in encoded_dict_sentiment.items()}
    
    if isinstance(id, list):
        return [encoded_dict_inv_sentiment[i] for i in id]
    else:
        return encoded_dict_inv_sentiment[id]

In [ ]:
encoded_dict_info = {'INFO':0, 'NOINFO':1, 'UNPREDICTABLE':2}

df['info_label'] = df['info'].apply(lambda x:encoded_dict_info[x])

def label2id_info(label):
    if isinstance(label, list):
        return [encoded_dict_info[label] for label in label]
    else:
        return encoded_dict_info[label]

def id2label_info(id):
    encoded_dict_inv_info = {v: k for k, v in encoded_dict_info.items()}
    
    if isinstance(id, list):
        return [encoded_dict_inv_info[i] for i in id]
    else:
        return encoded_dict_inv_info[id]

## 1.3 Train, Validation, Test split

In [ ]:
df_train, df_testval = train_test_split(df, test_size=0.25)

df_test, df_val = train_test_split(df_testval, test_size=0.2)

In [ ]:
print("Dimension of train dataset is:", df_train.shape)
print("Dimension of validation dataset is:", df_val.shape)
print("Dimension of test dataset is:", df_test.shape)

In [ ]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)
val_dataset = Dataset.from_pandas(df_val)

# Create the DatasetDict
dataset = DatasetDict({'train': train_dataset, 'test': test_dataset, 'validation': val_dataset})

print(dataset)

# 2. Preprocess data

In [ ]:
bert = TFAutoModel.from_pretrained('Twitter/twhin-bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('Twitter/twhin-bert-base')

In [ ]:
max_length = 256

def tokenize_text(dataset):
    return tokenizer(
        text=dataset['text'],
        add_special_tokens=True,
        return_token_type_ids=False,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='tf',
        verbose=True
    )


In [ ]:
encoded_dataset = dataset.map(tokenize_text)
encoded_dataset

In [ ]:
encoded_dataset = encoded_dataset.remove_columns(['sentiment','emotion','text','info','ID','DATE','CHANNEL','__index_level_0__'])

encoded_dataset

In [ ]:
def preprocess_data(encoded_dataset, data_type):
    input_ids = np.array(encoded_dataset[data_type]['input_ids'])
    input_ids = np.squeeze(input_ids, axis=1)

    attention_mask = np.array(encoded_dataset[data_type]['attention_mask'])
    attention_mask = np.squeeze(attention_mask, axis=1)

    sentiment_label = np.array(encoded_dataset[data_type]['sentiment_label'])
    emotion_label = np.array(encoded_dataset[data_type]['emotion_label'])
    info_label = np.array(encoded_dataset[data_type]['info_label'])

    return input_ids, attention_mask, sentiment_label, emotion_label, info_label

def main_processing(encoded_dataset):
    input_ids_train, attention_mask_train, sentiment_label_train, emotion_label_train, info_label_train = preprocess_data(encoded_dataset, 'train')
    input_ids_val, attention_mask_val, sentiment_label_val, emotion_label_val, info_label_val = preprocess_data(encoded_dataset, 'validation')
    input_ids_test, attention_mask_test, sentiment_label_test, emotion_label_test, info_label_test = preprocess_data(encoded_dataset, 'test')

    return input_ids_train, attention_mask_train, sentiment_label_train, emotion_label_train, info_label_train, input_ids_val, attention_mask_val, sentiment_label_val, emotion_label_val, info_label_val, input_ids_test, attention_mask_test, sentiment_label_test, emotion_label_test, info_label_test

# Usage
input_ids_train, attention_mask_train, sentiment_label_train, emotion_label_train, info_label_train, input_ids_val, attention_mask_val, sentiment_label_val, emotion_label_val, info_label_val, input_ids_test, attention_mask_test, sentiment_label_test, emotion_label_test, info_label_test = main_processing(encoded_dataset)

# 3. Defining the model

In [ ]:
input_ids = Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask")

embeddings = bert.bert(input_ids, attention_mask = input_mask)[0]
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.2)(out)
out = Dense(32, activation='relu')(out)

emotion_out = Dense(7, activation='softmax', name='emotion_output')(out)
sentiment_out = Dense(3, activation='softmax', name='sentiment_output')(out)
info_out = Dense(3, activation='softmax', name='info_output')(out)
    
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=[emotion_out,sentiment_out,info_out])
model.layers[2].trainable = True

In [ ]:
for i, layer in enumerate(model.layers):
    print(f"Layer {i}: {layer.name}")

In [ ]:
optimizer = AdamW(
    learning_rate=2e-5,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08,
    amsgrad=False,
    weight_decay=0.01,
    clipnorm=1.0,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name="AdamW"
)

In [ ]:
loss = SparseCategoricalCrossentropy(
    from_logits=False,
    ignore_class=None,
    reduction="auto",
    name="sparse_categorical_crossentropy",
)

In [ ]:
early_stop = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=2,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0,
)

In [ ]:
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=["sparse_categorical_accuracy"]
)

In [ ]:
model.summary()

# 4. Training

In [ ]:
# Assuming you have separate label arrays for each task: label_emotion_train, label_sentiment_train, label_info_train, label_emotion_val, label_sentiment_val, label_info_val

history = model.fit(
    x={'input_ids': input_ids_train, 'attention_mask': attention_mask_train},
    y={'emotion_output': emotion_label_train, 'sentiment_output': sentiment_label_train, 'info_output': info_label_train},
    validation_data=({'input_ids': input_ids_val, 'attention_mask': attention_mask_val},
                     {'emotion_output': emotion_label_val, 'sentiment_output': sentiment_label_val, 'info_output': info_label_val}),
    epochs=10,
    batch_size=16,
    callbacks=[early_stop]
)

In [ ]:
import matplotlib.pyplot as plt

# Plot the loss for each task
for task in ['emotion_output', 'sentiment_output', 'info_output']:
    plt.plot(history.history[f'{task}_loss'], label=f'train_{task}_loss')
    plt.plot(history.history[f'val_{task}_loss'], label=f'val_{task}_loss')

plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()


# 5. Metrics All

In [ ]:
# Make predictions for each task
predicted_emotion = model.predict({'input_ids': input_ids_test, 'attention_mask': attention_mask_test})[0]
predicted_sentiment = model.predict({'input_ids': input_ids_test, 'attention_mask': attention_mask_test})[1]
predicted_info = model.predict({'input_ids': input_ids_test, 'attention_mask': attention_mask_test})[2]

# For each task, get the predicted labels
predicted_labels_emotion = np.argmax(predicted_emotion, axis=1)
predicted_labels_sentiment = np.argmax(predicted_sentiment, axis=1)
predicted_labels_info = np.argmax(predicted_info, axis=1)

In [ ]:
predicted_labels_sentiment = predicted_labels_sentiment.tolist()
predicted_labels_sentiment  = id2label_sentiment(predicted_labels_sentiment)

predicted_labels_emotion = predicted_labels_emotion.tolist()
predicted_labels_emotion = id2label_emotion(predicted_labels_emotion)

predicted_labels_info = predicted_labels_info.tolist()
predicted_labels_info = id2label_info(predicted_labels_info)



emotion_label_test = emotion_label_test.tolist()
emotion_label_test = id2label_emotion(emotion_label_test)

sentiment_label_test = sentiment_label_test.tolist()
sentiment_label_test  = id2label_sentiment(sentiment_label_test)

info_label_test = info_label_test.tolist()
info_label_test = id2label_info(info_label_test)

In [ ]:
# Calculate classification reports for each task
report_emotion = classification_report(emotion_label_test, predicted_labels_emotion)
report_sentiment = classification_report(sentiment_label_test, predicted_labels_sentiment)
report_info = classification_report(info_label_test, predicted_labels_info)

# Print the reports
print("Emotion Classification Report:")
print(report_emotion)

print("\nSentiment Classification Report:")
print(report_sentiment)

print("\nInfo Classification Report:")
print(report_info)


## 5.1 Metrics Sentiment

In [ ]:
predicted_sentiment = model.predict({'input_ids': input_ids_test, 'attention_mask': attention_mask_test})
predicted_labels_sentiment = np.argmax(predicted_sentiment, axis=1)

In [ ]:
predicted_labels_sentiment = predicted_labels_sentiment.tolist()
predicted_labels_sentiment  = id2label_sentiment(predicted_labels_sentiment)

sentiment_label_test = sentiment_label_test.tolist()
sentiment_label_test  = id2label_sentiment(sentiment_label_test)

In [ ]:
report_sentiment = classification_report(sentiment_label_test, predicted_labels_sentiment)

print("Sentiment Classification Report:")
print(report_sentiment)

## 5.2 Metrics Emotion

In [ ]:
predicted_emotion = model.predict({'input_ids': input_ids_test, 'attention_mask': attention_mask_test})
predicted_labels_emotion = np.argmax(predicted_emotion, axis=1)

In [ ]:
predicted_labels_emotion = predicted_labels_emotion.tolist()
predicted_labels_emotion = id2label_emotion(predicted_labels_emotion)

emotion_label_test = emotion_label_test.tolist()
emotion_label_test = id2label_emotion(emotion_label_test)

In [ ]:
report_emotion = classification_report(emotion_label_test, predicted_labels_emotion)

print("Emotion Classification Report:")
print(report_emotion)

# 6. Push to the Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import push_to_hub_keras

push_to_hub_keras(model, 'FedeBerto/Griffith')

In [ ]:
from huggingface_hub import from_pretrained_keras

model = from_pretrained_keras('FedeBerto/Griffith-Emotion')